In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import emcee
import corner
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import WMAP9 as cosmo
from astropy.io import fits as pyfits

La distancia módulo es una forma de parametrizar distancias en el contexto de astronomía y cosmología mediante el uso de escalas logarítmicas. Cuando tratamos con supernovas, tenemos tres parámetros de interés: su color, su forma y su entorno galáctico. Si consideramos que las supernovas con parámetros idénticos presentan en promedio la misma luminosidad intrínsica para todos los corrimientos al rojo, podemos modelar la distancia módulo como

$\mu = m_{B}^{*} - (M_{B} - \alpha \times X_{1} + \beta \times C)$

donde $m_{B}^{*}$ es el pico de la magnitud de la banda $B$, $X_{1}$ es la extensión de la curvatura de luz y $C$ es el color de la supernova en su brillo máximo. Por su parte, $M_{B}$, $\alpha$ y $\beta$ no son otra cosa más que parámetros de ajuste que deben determinarse para que el modelo lineal concuerde con la estandarizaión $\mu = 5 log_{10}(d_{L}/10 pc)$, donde $d_{L}$ representa la distancia luminosa. De acuerdo a la teoría, es posible obtener esta cantidad para un objeto astronómico si conocemos su corrimiento al rojo. Usamos la libreria FlatLambdaCDM con los parámetros del universo actual (constante de Hubble $H_{0} = 70 km s^{-1} Mpc^{-1}$, parámetro de densidad de materia bariónica $\Omega_{0} = 0.3$ y temperatura del universo $T = 2.725 K$) para obtener los valores $d_{L}$ necesarios

In [2]:
datos = np.loadtxt('data/jla_lcparams.txt', usecols = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19))

In [3]:
z = np.zeros(740)

for i in range(740):
    z[i] = datos[i][0]

In [4]:
mb = np.zeros(740)
x1 = np.zeros(740)
c = np.zeros(740)

for i in range(740):
    mb[i] = datos[i][3]
    x1[i] = datos[i][5]
    c[i] = datos[i][7]

In [5]:
err_mb = np.zeros(740)
err_x1 = np.zeros(740)
err_c = np.zeros(740)

for i in range(740):
    err_mb[i] = datos[i][4]
    err_x1[i] = datos[i][6]
    err_c[i] = datos[i][8]

In [6]:
masa_estelar = np.zeros(740)
conjunto = np.zeros(740)

for i in range(740):
    masa_estelar[i] = datos[i][9]
    conjunto[i] = datos[i][16]

Y obtenemos las distancias módulo

In [7]:
err_total = np.zeros(740)
par_masa = np.zeros(740)

for i in range(740):
    err_aux = err_mb[i]**2 + err_x1[i]**2 + err_c[i]**2
    err_total[i] = err_aux

for i in range(740):
    if masa_estelar[i] >= 10:
        par_masa[i] = 1
    else:
        par_masa[i] = 0

In [8]:
c_n = pyfits.open('covmat/C_stat.fits')['PRIMARY'].data + pyfits.open('covmat/C_cal.fits')['PRIMARY'].data + pyfits.open('covmat/C_model.fits')['PRIMARY'].data + pyfits.open('covmat/C_bias.fits')['PRIMARY'].data + pyfits.open('covmat/C_host.fits')['PRIMARY'].data + pyfits.open('covmat/C_dust.fits')['PRIMARY'].data + pyfits.open('covmat/C_pecvel.fits')['PRIMARY'].data + pyfits.open('covmat/C_nonia.fits')['PRIMARY'].data

In [9]:
a = np.zeros((740, 2220))
n = np.zeros ((2220, 1))
delta_kronecker = np.zeros((740, 2220, 3))

In [10]:
for i in range(740):
    n[3*i] = mb[i]
    n[3*i + 1] = x1[i]
    n[3*i + 2] = c[i]

In [11]:
for i in range(740):
    for j in range(2220):
        for k in range(3):
            if 3*i == j + k:
                delta_kronecker[i][j][k] = 1
            else:
                delta_kronecker[i][j][k] = 0

In [12]:
masa_aux = np.ones(740)
diag_1 = np.zeros((740, 740))
diag_2 = np.zeros((740, 740))
diag_3 = np.zeros((740, 740))

In [13]:
for i in range(740):
    for j in range(740):
        if i == j:
            diag_1[i][j] = ((0.0025)/(z[i]*math.log(10)))**2

In [14]:
for i in range(740):
    for j in range(740):
        if i == j:
            diag_2[i][j] = (0.055*z[i])**2

In [15]:
for i in range(740):
    for j in range(740):
        if i == j:
            if conjunto[i] == 3:
                diag_3[i][j] = (0.12)**2
            elif conjunto[i] == 1:
                diag_3[i][j] = (0.11)**2
            elif conjunto[i] == 2:
                diag_3[i][j] = (0.08)**2
            else:
                diag_3[i][j] = (0.11)**2

In [35]:
def log_likelihood_mod(theta):
    omega, alfa, beta, masa, delta = theta
    for i in range(740):
        for j in range(2220):
            a[i][j] = delta_kronecker[i][j][0] + alfa*delta_kronecker[i][j][1] + beta*delta_kronecker[i][j][2]
    model = np.matmul(a, n) - (masa*masa_aux).reshape(740, 1) + (delta*par_masa).reshape(740, 1)
    omega = abs(omega)
    d_l = FlatLambdaCDM(H0 = 70, Om0 = omega, Tcmb0 = 2.725).luminosity_distance(z)
    mu_stand = np.zeros(740)
    for i in range(740):
        mu_aux = 5*math.log10(d_l[i].value/10e-6)
        mu_stand[i] = mu_aux
    likelihood_aux = model - mu_stand.reshape(740, 1)
    c_cov = np.matmul(np.matmul(a, c_n), a.reshape(2220, 740)) + diag_1 + diag_2 + diag_3
    c_cov_inv = np.linalg.inv(c_cov)
    likelihood = np.matmul(np.matmul(likelihood_aux.reshape(1, 740), c_cov_inv), likelihood_aux)
    return likelihood

In [36]:
def log_prior(theta):
    omega, alfa, beta, masa, delta = theta
    if 0.0 < omega < 1.0 and 0.0 < alfa < 1.0 and 3 < beta < 4 and -20 < masa < -19 and -1.0 < delta < 0.0:
        prior = 0
    else:
        prior = -np.inf
    return prior

In [37]:
def log_probability(theta):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    log_prob = lp + log_likelihood_mod(theta)
    print(log_prob)
    return log_prob

In [38]:
pos = [0.3, 0.15, 3.0, -19.0, -0.0070] + 1e-4 * np.random.randn(10, 5)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability)
sampler.run_mcmc(pos, 1000, progress=True);

[[-1127583.00289472]]


  0%|          | 0/1000 [00:00<?, ?it/s]

[[-1124896.93464083]]


  0%|          | 1/1000 [00:19<5:28:22, 19.72s/it]

[[-1125352.02440434]]
[[-1125260.53134215]]


...

  3%|▎         | 26/1000 [29:41<31:44:17, 117.31s/it]

[[-1075618.34555935]]
[[-1104775.3483942]]
[[-1029140.88607536]]
[[-1076185.35575012]]
[[-1016200.31854105]]


  3%|▎         | 26/1000 [31:17<19:32:01, 72.20s/it] 

[[-1038252.01544738]]


ValueError: If you start sampling with a given log_prob, you also need to provide the current list of blobs at that position.